In [1]:
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import json
import pickle

## Settings here!
frame_size = 10
batch_size = 1000
## End settings

tqdm.pandas()

cuda = torch.device('cpu')
ratings = pd.read_csv('../../data/ml-20m/ratings.csv')
movies = pickle.load(open('../../data/infos_pca128.pytorch', 'rb'))

# credits: KnightofK9
ratings["rating"] = ratings["rating"].apply(lambda i: 2 * (i - 2.5))
users = ratings[["userId","movieId"]].groupby(["userId"]).size()
users = users[users >= frame_size + 1]

In [2]:
import h5py
import numpy as np
import torch
import pickle

h5 = h5py.File("../../data/test/test.hdf5", "w")
idx = 0
n_batch = 0

In [3]:
ratings = ratings.drop("timestamp", axis=1).set_index("userId").groupby('userId')

for i in movies.keys():
    movies[i] = movies[i].to(cuda)

In [4]:
user_dict = {}
def app(x):
    userid = x.index[0]
    if userid in user_dict:
        print(userid)
    
    user_dict[int(userid)] = x
    return x

In [5]:
ratings.progress_apply(app)
print()

1




In [6]:
for userid in tqdm(user_dict.keys()):
    x = user_dict[userid]
    grp = h5.create_group('u' + str(userid))
    grp.create_dataset('movies', data=x['movieId'].values.astype(np.int32))
    grp.create_dataset('ratings', data=x['rating'].values.astype(np.int8))

In [7]:
h5['u1']['movies'][1]

29